### A3.3.2. Lock Contention

$$
T_{\text{contended}} = T_{\text{work}} + n \cdot T_{\text{lock\_wait}}
$$

where $n$ is the number of threads contending for the same lock and $T_{\text{lock\_wait}}$ is the average wait time per acquisition.

**Explanation:**

**Lock contention** occurs when multiple threads compete to acquire the same lock, serializing what should be parallel work. High contention degrades throughput and can negate the benefit of additional cores.

**Contention Taxonomy:**

| Type | Description | Impact |
|------|-------------|--------|
| Uncontended | Single thread holds lock, no waiters | Low overhead (~20 ns) |
| Low contention | Occasional short waits | Acceptable |
| High contention | Many threads spin/block on one lock | Near-serial execution |
| Convoy | All threads synchronize at lock release | Total serialization |

**Mitigation Strategies:**

1. **Reduce critical section** — hold the lock for the minimum time necessary.
2. **Lock striping** — split one lock into many, each protecting a subset of data (e.g., `ConcurrentHashMap` uses per-bucket locks).
3. **Lock-free data structures** — use atomic compare-and-swap (CAS) instead of locks.
4. **Read-write locks** — allow concurrent readers, exclusive writers.
5. **Thread-local accumulation** — each thread writes to local storage, merge results after.

**The Universal Scalability Law (USL):**

$$C(n) = \frac{n}{1 + \sigma(n-1) + \kappa \, n(n-1)}$$

where $\sigma$ is the contention penalty and $\kappa$ is the coherence (crosstalk) penalty. As $n$ grows, throughput can actually *decrease* if $\kappa > 0$.

**Example:**

A shared counter incremented by 8 threads under a global lock runs slower than a single thread (each increment acquires/releases the lock). With thread-local counters merged at the end, contention drops to zero during the work phase.

In [ ]:
import threading
import time

INCREMENTS_PER_THREAD = 500_000


def contended_increment(shared_counter, lock, count):
    for _ in range(count):
        with lock:
            shared_counter[0] += 1


def local_increment(local_results, thread_index, count):
    local_sum = 0
    for _ in range(count):
        local_sum += 1
    local_results[thread_index] = local_sum


thread_counts = [1, 2, 4, 8]

print("Contended (global lock):")
for num_threads in thread_counts:
    shared_counter = [0]
    lock = threading.Lock()
    threads = [
        threading.Thread(target=contended_increment, args=(shared_counter, lock, INCREMENTS_PER_THREAD))
        for _ in range(num_threads)
    ]
    start = time.perf_counter()
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()
    elapsed = time.perf_counter() - start
    total_ops = num_threads * INCREMENTS_PER_THREAD
    print(f"  {num_threads} threads: {elapsed:.3f}s, count={shared_counter[0]:,}, ops/s={total_ops/elapsed:,.0f}")

print("\nUncontended (thread-local):")
for num_threads in thread_counts:
    local_results = [0] * num_threads
    threads = [
        threading.Thread(target=local_increment, args=(local_results, index, INCREMENTS_PER_THREAD))
        for index in range(num_threads)
    ]
    start = time.perf_counter()
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()
    elapsed = time.perf_counter() - start
    total_count = sum(local_results)
    total_ops = num_threads * INCREMENTS_PER_THREAD
    print(f"  {num_threads} threads: {elapsed:.3f}s, count={total_count:,}, ops/s={total_ops/elapsed:,.0f}")

**References:**

[📘 Gunther, N. (2007). *Guerrilla Capacity Planning.* Springer.](https://link.springer.com/book/10.1007/978-3-540-31010-5)

[📘 Herlihy, M. & Shavit, N. (2012). *The Art of Multiprocessor Programming (Revised 1st ed.).* Morgan Kaufmann.](https://www.elsevier.com/books/the-art-of-multiprocessor-programming/herlihy/978-0-12-397337-5)

---

[⬅️ Previous: Work Partitioning](./01_work_partitioning.ipynb) | [Next: Linux perf Tool ➡️](../04_Profiling/01_linux_perf_tool.ipynb)